In [1]:
import pandas as pd

import sys
sys.path.insert(1, r'C:\Users\comp\Documents\New York')
import geoTim

In [14]:
gj = geoTim.load_geojson(r'C:\Users\comp\Documents\chicago_crime\Boundaries - Community Areas (current)\geo_export_46a463aa-c8dd-4b16-b111-e865e7ebc83e.shp')

In [19]:
df = geoTim.geojson_to_df(gj)

In [20]:
df

,area,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len,geometry
0,0.0,35,35,0.0,0.0,DOUGLAS,0.0,4.600462e+07,31027.054510,POLYGON ((-87.60914087617894 41.84469250265398...
1,0.0,36,36,0.0,0.0,OAKLAND,0.0,1.691396e+07,19565.506153,POLYGON ((-87.59215283879394 41.81692934626684...
2,0.0,37,37,0.0,0.0,FULLER PARK,0.0,1.991670e+07,25339.089750,POLYGON ((-87.62879823733725 41.80189303368919...
3,0.0,38,38,0.0,0.0,GRAND BOULEVARD,0.0,4.849250e+07,28196.837157,"POLYGON ((-87.6067081256125 41.81681377057218,..."
4,0.0,39,39,0.0,0.0,KENWOOD,0.0,2.907174e+07,23325.167906,POLYGON ((-87.59215283879394 41.81692934626684...
...,...,...,...,...,...,...,...,...,...,...
72,0.0,74,74,0.0,0.0,MOUNT GREENWOOD,0.0,7.558429e+07,48665.130539,POLYGON ((-87.69645961375822 41.70714491233857...
73,0.0,75,75,0.0,0.0,MORGAN PARK,0.0,9.187734e+07,46396.419362,POLYGON ((-87.64215204651398 41.68508211967085...
74,0.0,76,76,0.0,0.0,OHARE,0.0,3.718356e+08,173625.984660,(POLYGON ((-87.83658087874365 41.9863961115912...
75,0.0,77,77,0.0,0.0,EDGEWATER,0.0,4.844999e+07,31004.830946,POLYGON ((-87.65455590025104 41.99816614970252...


In [21]:
from shapely.geometry import MultiPolygon

In [22]:
from geoalchemy2 import Geometry, WKTElement
import  sqlalchemy 

pg = open('db_credentials.txt')
db = pg.read()
pg.close()

def to_multipolygon(x):
    from shapely.geometry import MultiPolygon, Polygon
    if x.geom_type == 'Polygon':
        return MultiPolygon([x])
    elif x.geom_type == 'MultiPolygon':
        return x
    else:
        raise Exception('Needs to be either polygon or multipolygon')
    

df['geom'] = df['geometry'].apply(lambda x : WKTElement(to_multipolygon(x), srid=4326))
df.drop(columns = ['geometry'], inplace = True)

# load into db
engine = sqlalchemy.create_engine(db)
df.to_sql('community_areas', engine, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid= 4326)})